# Ordering


## Warning: This notebook will place live orders

Use a paper trading account (during market hours).


In [1]:
import pandas as pd
import yfinance as yf

from ib_async import *
import logging


if True:
    util.logToConsole(logging.DEBUG)


util.startLoop()

ib = IB()
ib.connect('192.168.178.81', 7497, clientId=14)


2024-08-26 14:39:35,382 ib_async.client INFO Connecting to 192.168.178.81:7497 with clientId 14...
2024-08-26 14:39:35,389 ib_async.client INFO Connected
2024-08-26 14:39:35,425 ib_async.client DEBUG <<< 178,20240826 14:39:33 CET
2024-08-26 14:39:35,429 ib_async.client DEBUG >>> 71,2,14,
2024-08-26 14:39:35,432 ib_async.client INFO Logged on to server version 178
2024-08-26 14:39:35,438 ib_async.client DEBUG <<< 15,1,DUA437695
2024-08-26 14:39:35,486 ib_async.client DEBUG <<< 9,1,10
2024-08-26 14:39:35,489 ib_async.client DEBUG <<< 4,2,-1,2104,Market data farm connection is OK:usfarm.nj,
2024-08-26 14:39:35,492 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm.nj
2024-08-26 14:39:35,497 ib_async.client DEBUG <<< 4,2,-1,2104,Market data farm connection is OK:usfuture,
2024-08-26 14:39:35,500 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfuture
2024-08-26 14:39:35,503 ib_async.client DEBUG <<< 4,2,-1,2104,Market dat

<IB connected to 192.168.178.81:7497 clientId=14>

### Get liquidation value of whole account

In [3]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DUA437695', tag='NetLiquidationByCurrency', value='1000003.5374', currency='BASE', modelCode='')]

In [49]:
portfolio_pos = []
for position in ib.positions():
    contract = position.contract
    position_type = 'LONG' if position.position > 0.0 else 'SHORT'
    #print(f"{contract.conId}, {contract.secType}, {position_type}, {contract.symbol}, {contract.localSymbol}, {contract.exchange}, {contract.currency}, {position.position:8.2f}, {position.avgCost:8.2f}")
    portfolio_pos.append(
        [
            contract.conId,
            contract.secType,
            position_type,
            contract.symbol,
            contract.localSymbol,
            contract.exchange,
            contract.currency,
            position.position,
            position.avgCost,
        ]
    )

portfolio_df = pd.DataFrame(portfolio_pos, columns=['conId', 'secType', 'position_type', 'symbol', 'localSymbol', 'exchange', 'currency', 'position', 'avgCost'])

del portfolio_pos, contract, position_type
portfolio_df

,conId,secType,position_type,symbol,localSymbol,exchange,currency,position,avgCost
0,53028906,STK,LONG,EOAN,EOAN,IBIS,EUR,100.0,12.62000
1,14079,STK,LONG,ALV,ALV,IBIS,EUR,100.0,275.03745
2,521962337,STK,LONG,IBC1,IBC1,GETTEX2,EUR,100.0,100.47021
3,68598660,STK,SHORT,BAYN,BAYN,IBIS,EUR,-100.0,27.55000


In [73]:
exchange_map = {
    'IBIS': 'DE',
    'GETTEX2': 'MU'
}

def yf_symbol(x):
    ex = x['exchange']
    ex_sym = x['exchange_sym']
    sym = x['symbol']
    loc_sym = x['localSymbol']

    if not pd.isna(ex_sym):
        return f"{sym}.{ex_sym}"
    else:
        return f"{sym}"



portfolio_df['exchange_sym'] = portfolio_df['exchange'].map(exchange_map, na_action='ignore')
portfolio_df['yf_sym'] = portfolio_df.apply(yf_symbol, axis=1)
portfolio_df

,conId,secType,position_type,symbol,localSymbol,exchange,currency,position,avgCost,exchange_sym,yf_sym
0,53028906,STK,LONG,EOAN,EOAN,IBIS,EUR,100.0,12.62000,DE,EOAN.DE
1,14079,STK,LONG,ALV,ALV,IBIS,EUR,100.0,275.03745,DE,ALV.DE
2,521962337,STK,LONG,IBC1,IBC1,GETTEX2,EUR,100.0,100.47021,MU,IBC1.MU
3,68598660,STK,SHORT,BAYN,BAYN,IBIS,EUR,-100.0,27.55000,DE,BAYN.DE


In [77]:
import datetime
import yfinance as yf
import time

periods = 10
dt_end = datetime.datetime.today()
# Define real-time interval:
#  - assume to display at least the number of sample points of the larger period
#  - this requires double the number of points to create the averaging
#  - plus considering non-trading days - yfinance returns only trading days, howevers
dt_data_start = dt_end - datetime.timedelta(days=periods*2)

stock_data_list = []
for symbol in portfolio_df['yf_sym'].to_list(): 
    time.sleep(1)
    try:
        # Grab sufficient stock data for averaging SMAs
        load_df = yf.download(
            symbol,
            start=dt_data_start.strftime('%Y-%m-%d'),
            end=dt_end.strftime('%Y-%m-%d'),
            progress=False,
        )

        assert load_df.shape[1] == 6 and load_df.shape[0] >= periods
        stock_data_list.append((symbol, load_df))

    except AssertionError:
        print(f"Download failed for symbol {symbol}.  Skipping...")

2024-08-26 18:49:32,184 yfinance DEBUG Entering download()
2024-08-26 18:49:32,192 yfinance DEBUG Disabling multithreading because DEBUG logging enabled
2024-08-26 18:49:32,262 yfinance DEBUG  Entering history()
2024-08-26 18:49:32,338 peewee DEBUG ('SELECT "t1"."key", "t1"."value" FROM "_kv" AS "t1" WHERE ("t1"."key" = ?) LIMIT ? OFFSET ?', ['EOAN.DE', 1, 0])
2024-08-26 18:49:32,362 yfinance DEBUG   Entering history()
2024-08-26 18:49:32,437 yfinance DEBUG EOAN.DE: Yahoo GET parameters: {'period1': '2024-08-06 00:00:00+02:00', 'period2': '2024-08-26 00:00:00+02:00', 'interval': '1d', 'includePrePost': False, 'events': 'div,splits,capitalGains'}
2024-08-26 18:49:32,603 yfinance DEBUG EOAN.DE: yfinance received OHLC data: 2024-08-06 07:00:00 -> 2024-08-23 07:00:00
2024-08-26 18:49:32,673 yfinance DEBUG EOAN.DE: OHLC after cleaning: 2024-08-06 09:00:00+02:00 -> 2024-08-23 09:00:00+02:00
2024-08-26 18:49:32,754 yfinance DEBUG EOAN.DE: OHLC after combining events: 2024-08-06 00:00:00+02:00

In [78]:
for (bla, cs) in stock_data_list:
    print(bla)

EOAN.DE
ALV.DE
IBC1.MU
BAYN.DE


In [86]:
cs.shape

(14, 6)

In [84]:
cs['Adj Close'].iloc[-1]

27.864999771118164

In [96]:
from importlib import reload  # Python 3.4+
import utils.atr_utils as atr

reload(atr)

bla = atr.calc_atr_spikes(cs, periods=14)


[(Timestamp('2024-08-06 00:00:00'), 1.669999122619629), (Timestamp('2024-08-20 00:00:00'), 0.9249992370605469)]

    Arithmetic mean :      1.3
    Harmonic mean   :     1.19
    Geometric mean  :     1.24
    


In [ ]:
ib.disconnect()

2024-08-26 16:29:58,720 ib_async.ib INFO Disconnecting from 192.168.178.81:7497, 120 B sent in 8 messages, 227 kB received in 4840 messages, session time 6.62 ks.
2024-08-26 16:29:58,726 ib_async.client INFO Disconnecting


2024-08-26 16:29:58,756 ib_async.client INFO Disconnected.


Create a contract and a market order:

In [ ]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)

order = LimitOrder('SELL', 20000, 1.11, conditionsIgnoreRth=True)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [ ]:
trade = ib.placeOrder(contract, order)

### List orders and cancel an order

In [ ]:
orders = ib.openOrders()
orders

In [ ]:
orders

In [ ]:
ib.cancelOrder(order=orders[1] )

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [ ]:
ib.sleep(1)
trade.log

``trade`` will also available from ``ib.trades()``:

In [ ]:
assert trade in ib.trades()

Likewise for ``order``:

In [ ]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [ ]:
limitOrder = LimitOrder('BUY', 20000, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

``status`` will change from "PendingSubmit" to "Submitted":

In [ ]:
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

In [ ]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [ ]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

And now cancel it:

In [ ]:
ib.cancelOrder(limitOrder)

In [ ]:
limitTrade.log

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [ ]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

What are our positions?

In [ ]:
ib.positions()

What's the total of commissions paid today?

In [ ]:
sum(fill.commissionReport.commission for fill in ib.fills())

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [ ]:
order = MarketOrder('SELL', 20000)
ib.whatIfOrder(contract, order)

In [ ]:
ib.disconnect()